# **Deep Hedging**
# Buchkov Viacheslav

In [1]:
!pip install stable_baselines3
!pip install sb3-contrib


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import warnings
from pathlib import Path

from deep_hedging import ExperimentConfig, EuropeanCall, seed_everything
from deep_hedging.rl import DerivativeEnvStep, RLTrainer

from sb3_contrib import RecurrentPPO
from stable_baselines3 import SAC, PPO

RANDOM_SEED = 12

In [3]:
config = ExperimentConfig(
    RANDOM_SEED=RANDOM_SEED, DATA_ROOT=Path("data"), OUTPUT_ROOT=Path(".")
)
seed_everything(RANDOM_SEED)

config.DEVICE

device(type='mps')

In [4]:
config

ExperimentConfig(N_DAYS=5, REBAL_FREQ='30 min', VAR_QUANTILE=0.05, TEST_SIZE=0.1, RANDOM_SEED=12, DATA_ROOT=PosixPath('data'), OUTPUT_ROOT=PosixPath('.'), DATA_FILENAME='data_full', NORMALIZE=False, USE_TIME_DIFF=True, USE_SPOT_START=True, N_EPOCHS=20, N_STEPS_RL_TRAIN=1000000, LR=0.01, BATCH_SIZE=32, NUM_LAYERS=3, HIDDEN_DIM=32, OPTIMIZER=<class 'torch.optim.adam.Adam'>, NUM_WORKERS=2)

## Environment.

In [5]:
env = DerivativeEnvStep(n_days=config.N_DAYS, instrument_cls=EuropeanCall)
env.reset()

array([9.02507500e+01, 9.02802500e+01, 6.49158999e-02, 5.80593546e-02,
       0.00000000e+00])

## Training.

In [6]:
warnings.filterwarnings("ignore", category=UserWarning)

trainer = RLTrainer(
    model=RecurrentPPO("MlpLstmPolicy", env, verbose=1),
    instrument_cls=EuropeanCall,
    environment_cls=DerivativeEnvStep,
    config=config,
)
trainer.learn(10_000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------
| time/              |      |
|    fps             | 3112 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 128  |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 348         |
|    iterations           | 2           |
|    time_elapsed         | 0           |
|    total_timesteps      | 256         |
| train/                  |             |
|    approx_kl            | 0.004131758 |
|    clip_fraction        | 0.00469     |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.42       |
|    explained_variance   | -5.02       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.00328     |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.00283    |
|    std                

In [7]:
trainer.assess(3_000)

[array([4.611951], dtype=float32)]